In [132]:
import numpy as np
import pandas as pd

In [133]:
data = pd.read_csv('./datasets/Bitcoin Historical Data(2010-2022).csv')
data

,Date,Open,High,Low,Close,Volume
0,2010-07-18,0.0,0.1,0.1,0.1,75
1,2010-07-19,0.1,0.1,0.1,0.1,574
2,2010-07-20,0.1,0.1,0.1,0.1,262
3,2010-07-21,0.1,0.1,0.1,0.1,575
4,2010-07-22,0.1,0.1,0.1,0.1,2160
...,...,...,...,...,...,...
4301,2022-04-27,38113.0,39447.0,37869.0,39243.0,426830240
4302,2022-04-28,39243.0,40382.0,38888.0,39748.0,542391872
4303,2022-04-29,39748.0,39919.0,38175.0,38581.0,382904352
4304,2022-04-30,38590.0,38776.0,37596.0,37650.0,248220336


In [134]:
from datetime import datetime

data.columns = [d.lower() for d in data]

In [135]:
data.isna().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [136]:
data.date = np.array([datetime.strptime(d, '%Y-%m-%d') for d in data.date])

In [137]:
data.sort_values(by='date', inplace=True)
data

,date,open,high,low,close,volume
0,2010-07-18,0.0,0.1,0.1,0.1,75
1,2010-07-19,0.1,0.1,0.1,0.1,574
2,2010-07-20,0.1,0.1,0.1,0.1,262
3,2010-07-21,0.1,0.1,0.1,0.1,575
4,2010-07-22,0.1,0.1,0.1,0.1,2160
...,...,...,...,...,...,...
4301,2022-04-27,38113.0,39447.0,37869.0,39243.0,426830240
4302,2022-04-28,39243.0,40382.0,38888.0,39748.0,542391872
4303,2022-04-29,39748.0,39919.0,38175.0,38581.0,382904352
4304,2022-04-30,38590.0,38776.0,37596.0,37650.0,248220336


In [138]:
data = data[['date', 'open']]
data.rename(columns={'open': 'price'}, inplace=True)
data

/tmp/ipykernel_23903/1618604397.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'open': 'price'}, inplace=True)


,date,price
0,2010-07-18,0.0
1,2010-07-19,0.1
2,2010-07-20,0.1
3,2010-07-21,0.1
4,2010-07-22,0.1
...,...,...
4301,2022-04-27,38113.0
4302,2022-04-28,39243.0
4303,2022-04-29,39748.0
4304,2022-04-30,38590.0


In [139]:
data = data[data.date >= datetime(2011, 12, 30)]
data = data[data.date <= datetime(2019, 11, 24)]
data.reset_index(drop=True, inplace=True)
data

,date,price
0,2011-12-30,4.2
1,2011-12-31,4.3
2,2012-01-01,4.7
3,2012-01-02,5.3
4,2012-01-03,5.2
...,...,...
2882,2019-11-20,8145.7
2883,2019-11-21,8099.4
2884,2019-11-22,7627.9
2885,2019-11-23,7271.6


In [140]:
print(data.date.min())
print(data.date.max())
print((data.date.max() - data.date.min()).days)

2011-12-30 00:00:00
2019-11-24 00:00:00
2886


In [152]:
def normalize_price():
    x = 0
    price_previous_day = 0
    price_history = np.array([])

    for index, p in enumerate(data.price):
        if x == 0:
            price_history = np.append(price_history, 0)
            price_previous_day, index_before = p, index
            x += 1
            continue

        diff = p - price_previous_day
        price_previous_day = p

        if diff < 0:
            price_history = np.append(price_history, -1)
        elif diff == 0:
            price_history = np.append(price_history, 0)
        else:
            price_history = np.append(price_history, 1)

    return price_history

In [153]:
data['price_change'] = normalize_price()
data

,date,price,price_change
0,2011-12-30,4.2,0.0
1,2011-12-31,4.3,1.0
2,2012-01-01,4.7,1.0
3,2012-01-02,5.3,1.0
4,2012-01-03,5.2,-1.0
...,...,...,...
2882,2019-11-20,8145.7,-1.0
2883,2019-11-21,8099.4,-1.0
2884,2019-11-22,7627.9,-1.0
2885,2019-11-23,7271.6,-1.0


In [157]:
a = 0
b = 0
c = 0

for index, p in enumerate(data.price_change):
    if (p == -1):
        a += 1
    elif (p == 0):
        b += 1
    else:
        c += 1

print("Decrease: {}".format(a))
print("Neutral: {}".format(b))
print("Increase: {}".format(c))

Decrease: 1263
Neutral: 121
Increase: 1503
